In [7]:
import pandas as pd
import re

In [2]:
dest_dir = '/data_sda/zhiyuan/backdoor_result/Qwenvl2_vqav2/'

In [3]:
df = pd.read_csv('/data_sda/zhiyuan/backdoor_result/backdoor_Qwenvl2_vqav2_asr.csv')
df

,Model,Clean Accuracy,Clean ASR,Part 1 Acc,Part 1 ASR,Part 2 Acc,Part 2 ASR,Backdoor ASR
0,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_0,70.11,0.28,70.03,0.83,69.50,4.00,88.17
1,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_1_0,71.28,0.56,70.88,9.56,71.02,4.33,75.33
2,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_1,71.50,0.28,71.46,1.89,71.05,1.56,60.67
3,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_2,71.50,0.28,71.46,1.89,71.05,1.56,60.67
4,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_1,69.00,0.28,68.63,0.83,68.65,2.17,59.50
...,...,...,...,...,...,...,...,...
59,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,68.67,0.28,68.80,0.83,68.64,1.67,69.33
60,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_1_uni...,69.39,0.28,69.64,0.83,68.96,0.83,8.00
61,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,71.39,0.28,70.35,2.00,70.02,1.78,71.00
62,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_1_uni...,71.56,0.28,71.06,1.89,70.91,1.44,34.00


In [6]:
sorted = df.sort_values(by=['Backdoor ASR', 'Clean Accuracy'], ascending=[False, False])
sorted.reset_index(drop=True, inplace=True)
sorted.head(10)

,Model,Clean Accuracy,Clean ASR,Part 1 Acc,Part 1 ASR,Part 2 Acc,Part 2 ASR,Backdoor ASR
0,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,69.17,0.28,69.49,2.22,69.28,2.89,91.00
1,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_0,70.11,0.28,70.03,0.83,69.50,4.00,88.17
2,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,69.72,0.28,69.46,1.78,69.87,2.44,85.56
3,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,68.72,0.28,68.63,0.83,68.72,2.83,77.00
4,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_1_0,71.28,0.56,70.88,9.56,71.02,4.33,75.33
5,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,70.50,0.28,70.20,1.56,69.90,1.44,71.56
6,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,71.39,0.28,70.35,2.00,70.02,1.78,71.00
7,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,68.67,0.28,68.80,0.83,68.64,1.67,69.33
8,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_1_uni...,69.44,0.28,69.64,0.83,69.26,1.50,64.17
9,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,71.22,0.33,70.74,4.11,71.36,2.22,63.33


In [15]:
train_info = {}
# extract epoch, lr, batch size, gradient_accumulation_steps
for model in sorted['Model']:
    with open(f'{dest_dir}/{model}/README.md', 'r') as f:
        content = f.read()
        match = re.search(r'learning_rate:\s*(.*)', content)
        if match:
            lr = match.group(1)
        else:
            lr = 'N/A'
        match = re.search(r'train_batch_size:\s*(.*)', content)
        if match:
            batch_size = match.group(1)
        else:
            batch_size = 'N/A'
        match = re.search(r'num_epochs:\s*(.*)', content)
        if match:
            epoch = match.group(1)
        else:
            epoch = 'N/A'
        match = re.search(r'gradient_accumulation_steps:\s*(.*)', content)
        if match:
            gradient_accumulation_steps = match.group(1)
        else:
            gradient_accumulation_steps = 'N/A'
        train_info[model] = (lr, batch_size, epoch, gradient_accumulation_steps)

In [16]:
# add train info to the dataframe
sorted['lr'] = sorted['Model'].map(lambda x: train_info[x][0])
sorted['batch_size'] = sorted['Model'].map(lambda x: train_info[x][1])
sorted['epoch'] = sorted['Model'].map(lambda x: train_info[x][2])
sorted['gradient_accumulation_steps'] = sorted['Model'].map(lambda x: train_info[x][3])
sorted.head()

,Model,Clean Accuracy,Clean ASR,Part 1 Acc,Part 1 ASR,Part 2 Acc,Part 2 ASR,Backdoor ASR,lr,batch_size,epoch,gradient_accumulation_steps
0,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,69.17,0.28,69.49,2.22,69.28,2.89,91.00,5e-05,2,3.0,2
1,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_0,70.11,0.28,70.03,0.83,69.50,4.00,88.17,0.0001,2,2.0,2
2,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,69.72,0.28,69.46,1.78,69.87,2.44,85.56,5e-05,2,2.0,2
3,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,68.72,0.28,68.63,0.83,68.72,2.83,77.00,0.0001,4,3.0,4
4,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_1_0,71.28,0.56,70.88,9.56,71.02,4.33,75.33,3e-05,4,3.0,8
5,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,70.50,0.28,70.20,1.56,69.90,1.44,71.56,5e-05,2,2.0,4
6,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,71.39,0.28,70.35,2.00,70.02,1.78,71.00,2e-05,4,3.0,2
7,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,68.67,0.28,68.80,0.83,68.64,1.67,69.33,0.0001,2,3.0,8
8,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_1_uni...,69.44,0.28,69.64,0.83,69.26,1.50,64.17,5e-05,2,3.0,2
9,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,71.22,0.33,70.74,4.11,71.36,2.22,63.33,5e-05,4,1.0,4


In [18]:
# rearrange columns, move the last 4 columns to the second at start
cols = sorted.columns.tolist()
cols = cols[:1] + cols[-4:] + cols[1:-4]
sorted = sorted[cols]
sorted.head()

,Model,lr,batch_size,epoch,gradient_accumulation_steps,Clean Accuracy,Clean ASR,Part 1 Acc,Part 1 ASR,Part 2 Acc,Part 2 ASR,Backdoor ASR
0,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,5e-05,2,3.0,2,69.17,0.28,69.49,2.22,69.28,2.89,91.00
1,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_0,0.0001,2,2.0,2,70.11,0.28,70.03,0.83,69.50,4.00,88.17
2,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_2_uni...,5e-05,2,2.0,2,69.72,0.28,69.46,1.78,69.87,2.44,85.56
3,Qwenvl2-7b_vqav2_backdoor_multi_3_rate_0_2_uni...,0.0001,4,3.0,4,68.72,0.28,68.63,0.83,68.72,2.83,77.00
4,Qwenvl2-7b_vqav2_backdoor_multi_2_rate_0_1_0,3e-05,4,3.0,8,71.28,0.56,70.88,9.56,71.02,4.33,75.33


In [19]:
sorted.to_csv('/data_sda/zhiyuan/backdoor_result/backdoor_Qwenvl2_vqav2_asr_sorted.csv', index=False)